# Magic Square Puzzle

Lucerne University of Applied Sciences and Arts - School of Information Technology

A magic square is an arrangement of distinct integers in a square grid, such that the values in each row, in each
column and in the two main diagonals all add up to the same number. If n denotes the number of cells,
the values 1 to n are to be distributed.

Imports

In [35]:
from ortools.sat.python import cp_model
from itertools import product
import numpy as np

Define magic square size

In [36]:
n = 4
magic_constant = n * (n * n + 1) // 2
print(magic_constant)

34


Create model

In [37]:
model = cp_model.CpModel()

In [38]:
# model (create a square of n * n cells containing values from 1 to n^2)
board = [[model.NewIntVar(1, n*n, f"({i}, {j})") for j in range(1, n+1)] for i in range(1, n+1)]

# all numbers must be different, hence a number can only be used once
model.AddAllDifferent(list(np.concatenate(board)))
    
for i in range(n):
    model.Add(cp_model.LinearExpr.Sum([board[i][j] for j in range(n)]) == magic_constant)  # rows
    model.Add(cp_model.LinearExpr.Sum([board[j][i] for j in range(n)]) == magic_constant)  # columns

# diagonal top left to bottom right
model.Add(cp_model.LinearExpr.Sum([board[i][i] for i in range(n)]) == magic_constant)
# diagonal top right to bottom left
model.Add(cp_model.LinearExpr.Sum([board[i][n -1 - i] for i in range(n)]) == magic_constant)

In [39]:
# add constraints for online excercise
model.Add(board[0][0] == 9)
model.Add(board[0][n-1] == 8)
model.Add(board[n-1][0] == 6)
model.Add(board[n-1][n-1] == 11)

Callback for solution printing (adapt if you do not use an n*n board)

In [40]:
class SolutionPrinter(cp_model.CpSolverSolutionCallback):
    
    def __init__(self, variables):
        cp_model.CpSolverSolutionCallback.__init__(self)
        self.__variables = variables
        
    def on_solution_callback(self):
        for i in range(len(self.__variables)):
            for j in range(len(self.__variables)):
                print(f"[{self.Value(self.__variables[i][j])}] ", end='')
            print("\n")
        print("\n\n")

Solve and print all solutions

In [41]:
solver = cp_model.CpSolver()
status = solver.SearchForAllSolutions(model, SolutionPrinter(board))

[9] [16] [1] [8] 

[15] [2] [7] [10] 

[4] [13] [12] [5] 

[6] [3] [14] [11] 




[9] [16] [1] [8] 

[7] [10] [15] [2] 

[12] [5] [4] [13] 

[6] [3] [14] [11] 




[9] [14] [3] [8] 

[12] [4] [5] [13] 

[7] [15] [10] [2] 

[6] [1] [16] [11] 




[9] [1] [16] [8] 

[12] [4] [5] [13] 

[7] [15] [10] [2] 

[6] [14] [3] [11] 




[9] [15] [2] [8] 

[12] [1] [16] [5] 

[7] [4] [13] [10] 

[6] [14] [3] [11] 




[9] [15] [2] [8] 

[7] [1] [16] [10] 

[12] [4] [13] [5] 

[6] [14] [3] [11] 




[9] [14] [3] [8] 

[12] [1] [16] [5] 

[7] [4] [13] [10] 

[6] [15] [2] [11] 




[9] [14] [3] [8] 

[7] [1] [16] [10] 

[12] [4] [13] [5] 

[6] [15] [2] [11] 




[9] [3] [14] [8] 

[15] [2] [7] [10] 

[4] [13] [12] [5] 

[6] [16] [1] [11] 




[9] [3] [14] [8] 

[12] [13] [4] [5] 

[7] [16] [1] [10] 

[6] [2] [15] [11] 




[9] [3] [14] [8] 

[7] [13] [4] [10] 

[12] [16] [1] [5] 

[6] [2] [15] [11] 




[9] [3] [14] [8] 

[7] [10] [15] [2] 

[12] [5] [4] [13] 

[6] [16] [1] [11] 




[9] [2] [15] [8]

In [45]:
print(f"Branches:  {solver.ResponseStats()}")

Branches:  CpSolverResponse:
status: OPTIMAL
objective: 0
best_bound: 0
booleans: 24
conflicts: 47
branches: 89
propagations: 443
integer_propagations: 2072
walltime: 0.062759
usertime: 0.062759
deterministic_time: 0.000257711
primal_integral: 0

